## *Exercise 1: Web Scraping, Portfolio Analysis & Visualization*
### Objective: Scrape top stock gainers from Yahoo Finance, analyze their historical performance, construct a simulated portfolio, and visualize its risk and return.

### 1. Environment Setup:

In [2]:
%pip install webdriver-manager selenium pandas yfinance matplotlib seaborn beautifulsoup4 #%pip → es un magic command de Jupyter/VS Code Notebooks para instalar paquetes dentro del entorno del kernel activo.

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#%pip': Expected package name at the start of dependency specifier
    #%pip
    ^


In [3]:
# Herramientas de Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By # Indica el submódulo exacto dentro del paquete Selenium más rapido
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException


C:\Users\diego\AppData\Local\Temp\ipykernel_5504\1277127508.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### 2. Scrape Top 50 Gainers:

In [4]:
# Configuramos las opciones de Chrome para que no tenga probelmas.
chrome_options = Options()

# chrome_options.add_argument("--headless")
chrome_options.add_argument("--start-maximized") # Pantalla completa
chrome_options.add_argument("--lang=en-US") # Lenguaje inglés

# Iniciar el WebDriver de Chrome
# pasando nuestras 'chrome_options' como argumento.
driver =  webdriver.Chrome(options=chrome_options)

print("WebDriver iniciado con éxito.")

NameError: name 'Options' is not defined

In [ ]:
url = "https://finance.yahoo.com/markets/stocks/gainers" 
driver.get(url) # Abrir con , get() la URL en el navegador.
wait = WebDriverWait(driver, 15)
# 1) Localizar el botón "Rows per page"
btn = wait.until(EC.element_to_be_clickable(
    (By.XPATH, "//button[@aria-haspopup='listbox']"))
)
btn.click()

# 2) Esperar a que el menú esté visible
menu_container = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "div.dialog-container[aria-hidden='false']"))
)

# 3) Seleccionar la opción "50"
opt_50 = menu_container.find_element(By.CSS_SELECTOR, "[role='option'][data-value='50']")
driver.execute_script("arguments[0].click();", opt_50)

In [ ]:
# 1) Esperar el contenedor de la tabla
table_container = wait.until(
    EC.presence_of_element_located(
        (By.XPATH, "//*[@id='main-content-wrapper']/section[1]/div/div[2]/div")
    )
)

# 2) Dentro del contenedor, buscar todas las filas del <tbody>
rows = table_container.find_elements(By.XPATH, ".//table/tbody/tr")

data = []
for r in rows:
    try:
        symbol = r.find_element(By.XPATH, "./td[1]//a").text.strip()
    except:
        symbol = r.find_element(By.XPATH, "./td[1]").text.strip()
    try:
        name = r.find_element(By.XPATH, "./td[2]").text.strip()
    except:
        name = ""

    data.append((symbol, name))

# 3) Convertir a DataFrame
df = pd.DataFrame(data, columns=["Symbol", "Name"])


In [ ]:
df.to_csv("top_50_gainers.csv", index=False)

In [ ]:
df.head(49) # Verificar que tiene 50 filas

,Symbol,Name
0,AMBA,"Ambarella, Inc."
1,IREN,IREN Limited
2,BABA,Alibaba Group Holding Limited
3,AFRM,"Affirm Holdings, Inc."
4,DOOO,BRP Inc.
5,ADSK,"Autodesk, Inc."
6,CIFR,Cipher Mining Inc.
7,SATS,EchoStar Corporation
8,S,"SentinelOne, Inc."
9,EQX,Equinox Gold Corp.


In [ ]:
df.count()

Symbol    50
Name      50
dtype: int64

In [ ]:
import pandas as pd
import yfinance as yf
import pandas as pd
import yfinance as yf

# --- 1) Lista de símbolos desde tu df ---
tickers = (
    df['Symbol']
      .dropna()
      .astype(str)
      .str.upper()
      .drop_duplicates()
      .tolist()
)

# --- 2) Descargar datos (último año, frecuencia mensual) ---
raw = yf.download(
    " ".join(tickers),
    period="1y",
    interval="1mo",
    auto_adjust=False,
    group_by="ticker",
    threads=True,
    progress=False
)

# --- 3) Extraer 'Adj Close' en un solo DataFrame ancho ---
adj_list = []
for t in tickers:
    try:
        # Caso múltiple: raw con columnas multi-index
        s = raw[t]['Adj Close'].rename(t)
        adj_list.append(s)
    except Exception:
        try:
            # Si falló, descargar el ticker por separado
            s = yf.download(
                t, period="1y", interval="1mo",
                auto_adjust=False, progress=False
            )['Adj Close'].rename(t)
            adj_list.append(s)
        except:
            pass

# --- 4) Concatenar en DataFrame final ---
monthly_adj_close = pd.concat(adj_list, axis=1).sort_index()
monthly_adj_close.index.name = "Date"
monthly_adj_close = monthly_adj_close.tail(12)  # mantener solo últimos 12 meses

# --- 5) Resultado final ---
print("DataFrame final de 12 meses (Adj Close):")
print(monthly_adj_close.head())




DataFrame final de 12 meses (Adj Close):
                 AMBA   IREN        BABA       AFRM       DOOO        ADSK  \
Date                                                                         
2024-10-01  56.189999   9.12   96.351486  43.849998  48.756016  283.799988   
2024-11-01  71.550003  13.51   85.917831  70.010002  48.221645  291.899994   
2024-12-01  72.739998   9.82   83.380714  60.900002  50.339321  295.570007   
2025-01-01  76.720001  10.22   97.197182  61.070000  47.448830  311.339996   
2025-02-01  61.430000   8.24  130.307556  64.150002  39.350475  274.209991   

            CIFR       SATS          S   EQX  ...        KGC   FSM       BF-B  \
Date                                          ...                               
2024-10-01  4.93  25.059999  25.790001  5.54  ...   9.986554  4.97  43.180275   
2024-11-01  6.70  25.290001  27.950001  5.65  ...   9.619984  4.78  41.267910   
2024-12-01  4.64  22.900000  22.200001  5.02  ...   9.211755  4.29  37.247032   
2025-01

In [ ]:
monthly_adj_close.head(12)

,AMBA,IREN,BABA,AFRM,DOOO,ADSK,CIFR,SATS,S,EQX,...,KGC,FSM,BF-B,BTU,HP,WPM,UPWK,EMN,SNDK,TD
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-01,56.189999,9.120000,96.351486,43.849998,48.756016,283.799988,4.93,25.059999,25.790001,5.54,...,9.986554,4.97,43.180275,25.835382,32.161938,65.580757,13.560000,102.163750,NaN,52.644344
2024-11-01,71.550003,13.510000,85.917831,70.010002,48.221645,291.899994,6.70,25.290001,27.950001,5.65,...,9.619984,4.78,41.267910,23.455420,33.147858,61.924690,16.969999,101.804062,NaN,54.482735
2024-12-01,72.739998,9.820000,83.380714,60.900002,50.339321,295.570007,4.64,22.900000,22.200001,5.02,...,9.211755,4.29,37.247032,20.649555,30.877613,56.012188,16.350000,88.777184,NaN,51.311886
2025-01-01,76.720001,10.220000,97.197182,61.070000,47.448830,311.339996,5.73,27.660000,23.950001,6.07,...,11.209125,5.07,32.535694,17.898254,30.462954,62.216953,15.760000,97.698471,NaN,55.003178
2025-02-01,61.430000,8.240000,130.307556,64.150002,39.350475,274.209991,4.08,31.230000,20.629999,6.42,...,10.652643,4.32,32.634258,13.598727,25.564194,68.640823,15.930000,95.933723,NaN,58.522354
2025-03-01,50.330002,6.090000,130.032211,45.189999,33.554592,261.799988,2.30,25.580000,18.180000,6.88,...,12.530767,6.10,33.452332,13.425584,25.420986,77.315536,13.050000,86.384468,47.610001,58.551659
2025-04-01,47.990002,6.110000,117.444962,49.759998,33.633987,274.250000,2.85,22.480000,18.500000,6.70,...,14.706901,6.25,34.558651,12.226694,18.384472,83.181679,13.150000,76.190498,32.110001,62.302715
2025-05-01,52.639999,8.390000,111.947868,51.900002,43.857273,296.119995,3.12,17.730000,17.610001,6.63,...,14.696937,5.85,33.070766,13.039165,14.841885,86.582626,15.490000,77.546097,37.689999,68.335274
2025-06-01,66.070000,14.570000,111.525024,69.139999,48.312775,309.570007,4.78,27.700001,18.280001,5.75,...,15.605425,6.54,26.692690,13.365060,14.953461,89.800003,13.440000,73.875099,45.349998,72.689728


In [ ]:
monthly_adj_close.isna().sum()

AMBA     1
IREN     1
BABA     1
AFRM     1
DOOO     1
ADSK     1
CIFR     1
SATS     1
S        1
EQX      1
NXE      1
MIAX    11
NG       1
PRVA     1
SSRM     1
HL       1
CELH     1
HMY      1
OS       1
CNXC     1
BIDU     1
CDE      1
JOYY     1
GH       1
BTDR     1
COO      1
GFI      1
GSAT     1
SBSW     1
IAG      1
SOUN     1
HCC      1
NGD      1
BHC      1
BILI     1
AEM      1
SJM      1
BF-A     1
OLN      1
MOH      1
KGC      1
FSM      1
BF-B     1
BTU      1
HP       1
WPM      1
UPWK     1
EMN      1
SNDK     6
TD       1
dtype: int64

In [ ]:
import pandas as pd
import numpy as np

def select_portfolio_top_sharpe(monthly_adj_close: pd.DataFrame, n_assets: int = 10):
    """
    Selecciona n_assets por mayor Sharpe individual usando la PRIMERA MITAD
    de los retornos mensuales disponibles. Devuelve:
      - selected: lista de tickers seleccionados
      - weights: pd.Series con pesos equiponderados
      - table: DataFrame con métricas (media, vol, Sharpe) de la 1ra mitad
    """
    # 1) Retornos mensuales y limpieza
    rets = monthly_adj_close.sort_index().pct_change().dropna(how="all")
    # Si hay muy pocos meses, levanta error claro
    if rets.shape[0] < 4:
        raise ValueError("Se requieren al menos 5 meses de datos para una selección razonable.")

    # 2) Dividir en primera mitad (selección) y segunda mitad (validación futura)
    n = rets.shape[0]
    first_half = rets.iloc[:n//2, :].dropna(axis=1, how="any")  # exige datos completos en la 1ra mitad

    # Si por exigir datos completos te quedas con muy pocos, relaja a 'how="all"'
    if first_half.shape[1] < n_assets:
        first_half = rets.iloc[:n//2, :].dropna(axis=1, how="all")

    # 3) Métricas de selección (Sharpe simple, rf≈0)
    mean_ret = first_half.mean()
    vol_ret  = first_half.std()
    sharpe   = mean_ret / vol_ret.replace(0, np.nan)

    # 4) Top-n por Sharpe (descendente)
    ranked = sharpe.sort_values(ascending=False).dropna()
    selected = ranked.head(n_assets).index.tolist()

    # 5) Pesos equiponderados
    if len(selected) == 0:
        raise ValueError("No hay tickers suficientes con datos para calcular Sharpe.")
    w = pd.Series(1.0/len(selected), index=selected, name="weight")

    # 6) Tabla resumen
    table = pd.DataFrame({
        "MeanRet_1H": mean_ret.reindex(selected),
        "Vol_1H": vol_ret.reindex(selected),
        "Sharpe_1H": sharpe.reindex(selected),
        "Weight": w
    }).sort_values("Sharpe_1H", ascending=False)

    return selected, w, table

# ====== USO ======
selected, weights, selection_table = select_portfolio_top_sharpe(monthly_adj_close, n_assets=10)

# Mostrar los tickers seleccionados
print("Tickers seleccionados (Top-10 por Sharpe en 1ra mitad):")
print(selected)


C:\Users\diego\AppData\Local\Temp\ipykernel_18936\696516820.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


NameError: name 'monthly_adj_close' is not defined

In [ ]:
# --- Retornos mensuales individuales (últimos 6 meses, ya filtrado) ---
last6_ind_rets = (
    monthly_adj_close[selected]
    .sort_index()
    .pct_change()
    .dropna()
    .tail(6)
)


print("=== Retornos mensuales individuales (últimos 6m) ===")
print(last6_ind_rets)


NameError: name 'monthly_adj_close' is not defined

In [ ]:
# Pesos equiponderados (10% cada uno)
weights = pd.Series(1.0 / last6_ind_rets.shape[1], index=last6_ind_rets.columns)

# Retornos del portafolio = promedio ponderado de retornos individuales
port_rets_6m = last6_ind_rets.mul(weights, axis=1).sum(axis=1)

print("\n=== Retornos mensuales del portafolio (últimos 6m) ===")
print(port_rets_6m)


## Ejercicio II:Reddit API Data Collection & Sentiment Analysis
### Objective: Collect post and comment data from political subreddits using the Reddit API (PRAW), identify the most common posts and their comments


### 1. Enviroment Setup